# 1.0: Loading required packages:

In [ ]:
# Administrative stuff:
import os

# For Data manipulation:
import pandas as pd
import numpy as np
import random
import re

# Nice to haves
#import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# import googlemaps
import plotly.graph_objects as go

## dynamic text updating
#from IPython.display import Markdown as md

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [37]:
from plotly import graph_objects as go
import plotly.figure_factory as ff

# 1.1: Getting Data from the Cleaning File

In [38]:
%store -r air_data
%store -r air_ken
%store -r air_ken_2019
%store -r health_county_names
%store -r health_data
%store -r health_ken
%store -r health_ken_2019
%store -r health_ken_2019_clean
%store -r health_ken_2019_fm
%store -r so2
%store -r so2_county_names
%store -r so2_ken
%store -r airq_0
%store -r so2_ken_f
%store -r good_days_ken
%store -r daily_aqi
%store -r daily_aqi_ken
%store -r so2_ken_ff
%store -r so2_reshape

no stored variable or alias airq_0
no stored variable or alias so2_ken_ff


In [39]:
pd.set_option('display.max_columns',500)
pd.set_option('display.width',500)

# 1.2: Merging air and Health Data First

In [40]:
air_ken_2019.head(2)
health_ken_2019_fm.head(2)

,index,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,92.395437
1,327,Kentucky,Boone,2019,239,222,17,0,0,0,0,80,49,39,0,0,239,0,0,92.887029


,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,3596,2019,Adair,21001,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,44.8,"3,016",POINT (-85.15821669 37.13121962),21001970300,LPA
2,3598,2019,Anderson,21005,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,26.6,"5,372",POINT (-84.92306901 38.02049683),21005950201,DEPRESSION


In [41]:
health_ken_2019_fm[health_ken_2019_fm["CountyName"] == "Jefferson"]

,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
53,3649,2019,Jefferson,21111,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,6.1,"2,924",POINT (-85.78202288 38.27661851),21111000200,CANCER
55,3651,2019,Jefferson,21111,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,17.2,"2,447",POINT (-85.78253076 38.2614617),21111002300,ACCESS2
58,3654,2019,Jefferson,21111,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,Crude prevalence,15.9,"6,253",POINT (-85.52422538 38.234524),21111010402,CSMOKING
59,3655,2019,Jefferson,21111,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,21.3,"3,078",POINT (-85.64078951 38.24198879),21111010601,DEPRESSION
13421,825740,2019,Jefferson,21111,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,7.7,"5,222",POINT (-85.56243511 38.15331331),21111011519,ACCESS2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29453,841817,2019,Jefferson,21111,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,51.4,"2,540",POINT (-85.80521343 38.24337925),21111001000,LPA
29458,841822,2019,Jefferson,21111,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,10.9,"4,078",POINT (-85.57559354 38.25625142),21111010102,MHLTH
29459,841823,2019,Jefferson,21111,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,81.3,"6,220",POINT (-85.58511499 38.17750746),21111011110,CHECKUP
29465,841829,2019,Jefferson,21111,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,10.2,"1,949",POINT (-85.69251985 38.2569818),21111007601,PHLTH


In [42]:
chck = health_ken_2019_fm[health_ken_2019_fm["CountyName"].isin(air_ken_2019["County"])].groupby(["CountyName"]).agg(unique_geos = ("Geolocation", "nunique"), 
                                                                                                              cnts = ("Measure", "count")).reset_index()
sum(chck["cnts"])

6900

In [43]:
# Initial merge: 4282432 rows × 23 columns
# Next: 6768 rows 

# Merging:

merged_ah = pd.merge(air_ken_2019, health_ken_2019_fm, how = "left", left_on = "County", 
                     right_on = "CountyName",suffixes=('_air', '_health'))
merged_ah.head(2)

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,92.395437,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,92.395437,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH


In [44]:
air_ken_2019.shape
health_ken_2019_fm.shape
merged_ah.shape

(27, 20)

(13272, 13)

(6900, 33)

In [45]:
merged_ah["County"].value_counts()

Jefferson     2280
Fayette        984
Campbell       300
Warren         288
Daviess        276
Hardin         264
Boone          252
Pike           228
Christian      216
McCracken      204
Bullitt        204
Pulaski        168
Oldham         168
Boyd           156
Henderson      132
Bell           108
Jessamine      108
Greenup        108
Perry           96
Carter          84
Morgan          60
Simpson         48
Hancock         36
Edmonson        36
Trigg           36
Washington      36
Livingston      24
Name: County, dtype: int64

In [46]:
merged_ah[merged_ah["County"] == "Jefferson"]

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
3156,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,3649,2019,Jefferson,21111,Health Outcomes,Cancer (excluding skin cancer) among adults ag...,%,Crude prevalence,6.1,"2,924",POINT (-85.78202288 38.27661851),21111000200,CANCER
3157,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,3651,2019,Jefferson,21111,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,17.2,"2,447",POINT (-85.78253076 38.2614617),21111002300,ACCESS2
3158,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,3654,2019,Jefferson,21111,Health Risk Behaviors,Current smoking among adults aged >=18 years,%,Crude prevalence,15.9,"6,253",POINT (-85.52422538 38.234524),21111010402,CSMOKING
3159,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,3655,2019,Jefferson,21111,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,21.3,"3,078",POINT (-85.64078951 38.24198879),21111010601,DEPRESSION
3160,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,825740,2019,Jefferson,21111,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,7.7,"5,222",POINT (-85.56243511 38.15331331),21111011519,ACCESS2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,841817,2019,Jefferson,21111,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,51.4,"2,540",POINT (-85.80521343 38.24337925),21111001000,LPA
5432,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,841822,2019,Jefferson,21111,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,10.9,"4,078",POINT (-85.57559354 38.25625142),21111010102,MHLTH
5433,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,841823,2019,Jefferson,21111,Prevention,Visits to doctor for routine checkup within th...,%,Crude prevalence,81.3,"6,220",POINT (-85.58511499 38.17750746),21111011110,CHECKUP
5434,340,Kentucky,Jefferson,2019,365,200,163,2,0,0,0,136,70,49,0,10,164,191,0,54.794521,841829,2019,Jefferson,21111,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,10.2,"1,949",POINT (-85.69251985 38.2569818),21111007601,PHLTH


In [47]:
air_ken_2019[air_ken_2019["County"] == "Bell"]
health_ken_2019_clean[health_ken_2019_clean["CountyName"] == "Bell"]

,index,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,92.395437


,index,Year,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
3,3599,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.8,"5,150",POINT (-83.73909778 36.76433928),21013960200,STROKE
4,3600,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.5,"2,381",POINT (-83.61861461 36.71356868),21013960300,STROKE
5,3601,2019,Bell,21013,Health Outcomes,Stroke among adults aged >=18 years,%,Crude prevalence,5.3,"2,849",POINT (-83.72538629 36.61758576),21013960700,STROKE
191,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION
255,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13002,825231,2019,Bell,21013,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,23.3,"1,896",POINT (-83.53196181 36.73527758),21013960400,MHLTH
13103,825354,2019,Bell,21013,Health Outcomes,Diagnosed diabetes among adults aged >=18 years,%,Crude prevalence,16.9,"2,381",POINT (-83.61861461 36.71356868),21013960300,DIABETES
13152,825409,2019,Bell,21013,Health Outcomes,Coronary heart disease among adults aged >=18 ...,%,Crude prevalence,12.6,"2,564",POINT (-83.58378607 36.8540896),21013960100,CHD
13283,825572,2019,Bell,21013,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,14.1,"3,771",POINT (-83.76484451 36.64167333),21013960800,ACCESS2


In [48]:
merged_ah.head(2)

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId
0,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,92.395437,774350,2019,Bell,21013,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,30.6,"1,896",POINT (-83.53196181 36.73527758),21013960400,DEPRESSION
1,326,Kentucky,Bell,2019,263,243,20,0,0,0,0,77,49,39,0,0,234,29,0,92.395437,809745,2019,Bell,21013,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,25.1,"1,896",POINT (-83.53196181 36.73527758),21013960400,PHLTH


In [49]:
so2_ken.head(2)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
97694,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-01,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01
97695,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-02,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01


# 1.3: Merge Sulfur with Air and Health dataset

In [50]:
merged_ahs = pd.merge(merged_ah, so2_reshape, how = "inner", left_on = "County", right_on = "County Name",suffixes=('_ah', '_Sulfur'))
merged_ahs.head(2)

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId,County Name,Site Num,so2_lat,so2_long,so2_city_name,so2_n_dates,so2_good_days,so2_moderate_days,so2_unhealthy_days,so2_unhealthy_sens_days,so2_avg_aqi,so2_max_aqi,so2_min_AQI
0,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2,85.753425,764356,2019,Boyd,21019,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,15.4,"4,484",POINT (-82.63164476 38.44922764),21019030500,PHLTH,Boyd,17,38.45934,-82.64041,Ashland,365,365,0,0,0,0.479452,9.0,0.0
1,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2,85.753425,793478,2019,Boyd,21019,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,33.4,"7,919",POINT (-82.71349743 38.32089352),21019031100,LPA,Boyd,17,38.45934,-82.64041,Ashland,365,365,0,0,0,0.479452,9.0,0.0


In [51]:
merged_ah.shape
merged_ahs.shape

(6900, 33)

(9276, 46)

In [52]:
so2_ken_f.head(3)

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,Pollutant Standard,Date Local,Units of Measure,Event Type,Observation Count,Observation Percent,Arithmetic Mean,1st Max Value,1st Max Hour,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change,AQI_category,good_aqi,moderate_aqi,unhealthy_aqi,unhealthy_sens_aqi
97694,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-01,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good,True,False,False,False
97695,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-02,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good,True,False,False,False
97696,21,19,17,42401,1,38.45934,-82.64041,WGS84,Sulfur dioxide,1 HOUR,SO2 1-hour 2010,2019-01-03,Parts per billion,None,23,96.0,0.0,0.0,0,0.0,100.0,INSTRUMENTAL - ULTRAVIOLET FLUORESCENCE,ASHLAND PRIMARY (FIVCO),"FIVCO HEALTH DEPARTMENT, 2924 HOLT STREET",Kentucky,Boyd,Ashland,"Huntington-Ashland, WV-KY-OH",2021-11-01,Good,True,False,False,False


## Refining Air-Health-Sulfur Data:

In [53]:
# Making join more precise by limiting it to 15 miles distance between data collection points for health and sulfur:
# Creating required columns and adjusting type:
import re
merged_ahs['health_long'] = [re.split(' |\(|\)', x)[2] for x in merged_ahs.Geolocation]
merged_ahs['health_lat'] = [re.split(' |\(|\)', x)[3] for x in merged_ahs.Geolocation]
merged_ahs["health_long"] = merged_ahs["health_long"].astype(float)
merged_ahs["health_lat"] = merged_ahs["health_lat"].astype(float)
merged_ahs["so2_long"] = merged_ahs["so2_long"].astype(float)
merged_ahs["so2_lat"] = merged_ahs["so2_lat"].astype(float)
merged_ahs.head(n=3)

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956    # 6371 Radius of earth in kilometers. Use 3956 for miles
    return c * r

merged_ahs['dist'] = merged_ahs.apply(lambda row: haversine(row['health_long'], 
                                            row['health_lat'], 
                                            row['so2_long'], 
                                            row['so2_lat']), axis=1)

merged_ahs[['health_long', "health_lat", "so2_long", "so2_lat", "dist"]]

# Setting threhold to 15 miles
merged_ahs["precise"] = (merged_ahs["dist"] < 15)
# We can make it more precise by 1400 rows?
merged_ahs["precise"].value_counts()

,index_air,State,County,Year_air,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10,% of good days,index_health,Year_health,CountyName,CountyFIPS,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,TotalPopulation,Geolocation,LocationID,MeasureId,County Name,Site Num,so2_lat,so2_long,so2_city_name,so2_n_dates,so2_good_days,so2_moderate_days,so2_unhealthy_days,so2_unhealthy_sens_days,so2_avg_aqi,so2_max_aqi,so2_min_AQI,health_long,health_lat
0,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2,85.753425,764356,2019,Boyd,21019,Health Status,Physical health not good for >=14 days among a...,%,Crude prevalence,15.4,"4,484",POINT (-82.63164476 38.44922764),21019030500,PHLTH,Boyd,17,38.45934,-82.64041,Ashland,365,365,0,0,0,0.479452,9.0,0.0,-82.631645,38.449228
1,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2,85.753425,793478,2019,Boyd,21019,Health Risk Behaviors,No leisure-time physical activity among adults...,%,Crude prevalence,33.4,"7,919",POINT (-82.71349743 38.32089352),21019031100,LPA,Boyd,17,38.45934,-82.64041,Ashland,365,365,0,0,0,0.479452,9.0,0.0,-82.713497,38.320894
2,328,Kentucky,Boyd,2019,365,313,52,0,0,0,0,74,53,39,0,11,192,160,2,85.753425,809792,2019,Boyd,21019,Health Outcomes,Chronic obstructive pulmonary disease among ad...,%,Crude prevalence,11.7,"3,397",POINT (-82.64521256 38.46567548),21019030700,COPD,Boyd,17,38.45934,-82.64041,Ashland,365,365,0,0,0,0.479452,9.0,0.0,-82.645213,38.465675


,health_long,health_lat,so2_long,so2_lat,dist
0,-82.631645,38.449228,-82.64041,38.45934,0.843866
1,-82.713497,38.320894,-82.64041,38.45934,10.345064
2,-82.645213,38.465675,-82.64041,38.45934,0.508688
3,-82.686317,38.473947,-82.64041,38.45934,2.678826
4,-82.615679,38.330993,-82.64041,38.45934,8.962250
...,...,...,...,...,...
9271,-88.680700,37.022095,-88.57251,37.05822,6.463342
9272,-88.517810,37.009503,-88.57251,37.05822,4.517098
9273,-88.593599,37.071622,-88.57251,37.05822,1.485357
9274,-88.633469,37.077445,-88.57251,37.05822,3.611228


True     7812
False    1464
Name: precise, dtype: int64

# 1.4: Storing Final Data

In [54]:
%store merged_ahs
%store merged_ah

Stored 'merged_ahs' (DataFrame)
Stored 'merged_ah' (DataFrame)
